# Intermediate Cones

In [1]:
import numpy as np
import pypolycontain as pp
np.random.seed(0)
n,q_i,q_c=5,9,11
Z_i=pp.zonotope(x=np.zeros((n,1)),G=np.random.random((n,q_i))-0.5)
Z_c=pp.zonotope(x=np.zeros((n,1)),G=np.random.random((n,q_c))-0.5)

In [2]:
# Z_i.G=np.array([[ 1., -4., -3.],
#        [ 3.,  5.,  4.],
#        [-1.,  5., -4.]])
# Z_c.G=np.array([[-4., -4., -4.,  3.],
#        [-1., -1.,  5., -1.],
#        [-2.,  3., -2.,  1.]])
circumbody=pp.to_AH_polytope(Z_c)

In [3]:
pp.containment.extreme_rays_for_containment(circumbody,0)
K=circumbody.dim_complement
Theta={}
for k in range(-1,K+1):
    Theta[k]=pp.extreme_rays_for_containment(circumbody,k)
    print(k,Theta[k].shape)

-1 (22, 22)
0 (22, 671)
1 (22, 935)
2 (22, 935)
3 (22, 671)
4 (22, 341)
5 (22, 121)
6 (22, 22)


In [4]:
import pydrake.solvers.mathematicalprogram as MP
# use Gurobi solver
import pydrake.solvers.gurobi as Gurobi_drake
gurobi_solver=Gurobi_drake.GurobiSolver()
Theta={}
for i in range(-1,K+1):
    program=MP.MathematicalProgram()
    alpha=program.NewContinuousVariables(1,"alpha")
    parametric_circumbody=circumbody.copy()
    parametric_circumbody.P.h=circumbody.P.h*alpha
    Theta[i],*_=pp.subset(program,Z_i,parametric_circumbody,k=i)
    program.AddLinearCost(np.eye(1),np.zeros((1)),alpha)
    result=gurobi_solver.Solve(program,None,None)
    if result.is_success():
        print("%d: subset test successfull"%i)
        print(r"alpha=",1/result.GetSolution(alpha), "Theta size=",Theta[i].shape)
    else:
        print("not a subset") 

-1: subset test successfull
alpha= [0.41152581] Theta size= (22, 22)
0: subset test successfull
alpha= [0.41152581] Theta size= (22, 671)


KeyboardInterrupt: 